In [32]:
from datasets import load_dataset,concatenate_datasets

In [25]:
finance_qa_dataset_name = "gbharti/finance-alpaca"
text_2_sql_dataset_name = "Clinton/Text-to-sql-v1"
math_dataset_name = "math_qa"

In [3]:
text_2_sql_dataset = load_dataset(text_2_sql_dataset_name, split="train")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [4]:
text_2_sql_dataset[0]

{'instruction': 'Name the home team for carlton away team',
 'input': 'CREATE TABLE table_name_77 (\n    home_team VARCHAR,\n    away_team VARCHAR\n)',
 'response': 'SELECT home_team FROM table_name_77 WHERE away_team = "carlton"',
 'source': 'sql_create_context',
 'text': 'Below are sql tables schemas paired with instruction that describes a task. Using valid SQLite, write a response that appropriately completes the request for the provided tables. ### Instruction: Name the home team for carlton away team ### Input: CREATE TABLE table_name_77 (\n    home_team VARCHAR,\n    away_team VARCHAR\n) ### Response: SELECT home_team FROM table_name_77 WHERE away_team = "carlton"'}

In [6]:
text_2_sql_dataset[0]['text'].split("Response:")

['Below are sql tables schemas paired with instruction that describes a task. Using valid SQLite, write a response that appropriately completes the request for the provided tables. ### Instruction: Name the home team for carlton away team ### Input: CREATE TABLE table_name_77 (\n    home_team VARCHAR,\n    away_team VARCHAR\n) ### ',
 ' SELECT home_team FROM table_name_77 WHERE away_team = "carlton"']

In [7]:
text_2_sql_dataset = text_2_sql_dataset.map(
    lambda example: {"text": 
                     "<s>[INST] " + example["text"].split("Response")[0] + "[/INST]" + example["text"].split("Response")[1] + "</s>"},
    num_proc=4,
)

Map (num_proc=4):   0%|          | 0/262208 [00:00<?, ? examples/s]

In [8]:
text_2_sql_dataset = text_2_sql_dataset.remove_columns(["input", "instruction", "response","source"])

In [9]:
text_2_sql_dataset[0]

{'text': '<s>[INST] Below are sql tables schemas paired with instruction that describes a task. Using valid SQLite, write a response that appropriately completes the request for the provided tables. ### Instruction: Name the home team for carlton away team ### Input: CREATE TABLE table_name_77 (\n    home_team VARCHAR,\n    away_team VARCHAR\n) ### [/INST]: SELECT home_team FROM table_name_77 WHERE away_team = "carlton"</s>'}

In [19]:
math_dataset = load_dataset(math_dataset_name, split="train")

In [20]:
math_dataset[0]['Rationale'].split("answer")

['"explanation : t = 3 years r = 10 % td = ( bg × 100 ) / tr = ( 36 × 100 ) / ( 3 × 10 ) = 12 × 10 = rs . 120 td = ( pw × tr ) / 100 ⇒ 120 = ( pw × 3 × 10 ) / 100 ⇒ 1200 = pw × 3 pw = 1200 / 3 = rs . 400 ',
 ' : option a"']

In [21]:
math_dataset = math_dataset.map(
    lambda example: {"text": 
                     "<s>[INST] " + example["Problem"] + "[/INST]" + example["Rationale"].split("answer")[0] + "</s>"},
    num_proc=4,
)

In [22]:
math_dataset[0]

{'Problem': "the banker ' s gain of a certain sum due 3 years hence at 10 % per annum is rs . 36 . what is the present worth ?",
 'Rationale': '"explanation : t = 3 years r = 10 % td = ( bg × 100 ) / tr = ( 36 × 100 ) / ( 3 × 10 ) = 12 × 10 = rs . 120 td = ( pw × tr ) / 100 ⇒ 120 = ( pw × 3 × 10 ) / 100 ⇒ 1200 = pw × 3 pw = 1200 / 3 = rs . 400 answer : option a"',
 'options': 'a ) rs . 400 , b ) rs . 300 , c ) rs . 500 , d ) rs . 350 , e ) none of these',
 'correct': 'a',
 'annotated_formula': 'divide(multiply(const_100, divide(multiply(36, const_100), multiply(3, 10))), multiply(3, 10))',
 'linear_formula': 'multiply(n2,const_100)|multiply(n0,n1)|divide(#0,#1)|multiply(#2,const_100)|divide(#3,#1)|',
 'category': 'gain',
 'text': '<s>[INST] the banker \' s gain of a certain sum due 3 years hence at 10 % per annum is rs . 36 . what is the present worth ?[/INST]"explanation : t = 3 years r = 10 % td = ( bg × 100 ) / tr = ( 36 × 100 ) / ( 3 × 10 ) = 12 × 10 = rs . 120 td = ( pw × tr ) / 1

In [23]:
math_dataset = math_dataset.remove_columns(["Problem", "Rationale", "options","correct","annotated_formula","linear_formula", "category"])

In [24]:
math_dataset[0]

{'text': '<s>[INST] the banker \' s gain of a certain sum due 3 years hence at 10 % per annum is rs . 36 . what is the present worth ?[/INST]"explanation : t = 3 years r = 10 % td = ( bg × 100 ) / tr = ( 36 × 100 ) / ( 3 × 10 ) = 12 × 10 = rs . 120 td = ( pw × tr ) / 100 ⇒ 120 = ( pw × 3 × 10 ) / 100 ⇒ 1200 = pw × 3 pw = 1200 / 3 = rs . 400 </s>'}

In [26]:
finance_qa_dataset = load_dataset(finance_qa_dataset_name, split="train")

In [27]:
finance_qa_dataset = finance_qa_dataset.map(
    lambda example: {"text": 
                     "<s>[INST] " + example["instruction"] + "[/INST]" + example["output"] + "</s>"},
    num_proc=4,
)

In [28]:
finance_qa_dataset = finance_qa_dataset.remove_columns(["input", "instruction", "output"])

In [29]:
finance_qa_dataset[0]

{'text': "<s>[INST] For a car, what scams can be plotted with 0% financing vs rebate?[/INST]The car deal makes money 3 ways. If you pay in one lump payment. If the payment is greater than what they paid for the car, plus their expenses, they make a profit. They loan you the money. You make payments over months or years, if the total amount you pay is greater than what they paid for the car, plus their expenses, plus their finance expenses they make money. Of course the money takes years to come in, or they sell your loan to another business to get the money faster but in a smaller amount. You trade in a car and they sell it at a profit. Of course that new transaction could be a lump sum or a loan on the used car... They or course make money if you bring the car back for maintenance, or you buy lots of expensive dealer options. Some dealers wave two deals in front of you: get a 0% interest loan. These tend to be shorter 12 months vs 36,48,60 or even 72 months. The shorter length makes i

In [30]:
text_2_sql_dataset = text_2_sql_dataset.select(range(1000))
math_dataset = math_dataset.select(range(1000))
finance_qa_dataset = finance_qa_dataset.select(range(1000))

In [35]:
type(text_2_sql_dataset)

datasets.arrow_dataset.Dataset

In [38]:
temp_dataset = concatenate_datasets([text_2_sql_dataset, math_dataset])
final_dataset = concatenate_datasets([temp_dataset,finance_qa_dataset])

In [39]:
final_dataset.save_to_disk('final_dataset')

Saving the dataset (0/1 shards):   0%|          | 0/3000 [00:00<?, ? examples/s]